You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	JOIN Operation:
o	Perform a "JOIN" operation to combine information from the launches data and the rockets data.
o	Join Logic: For each launch record, look up the corresponding rocket record using the rocket identifier. The join key is the rocket.rocket_id field within each launch record, which matches the rocket_id field in the rocket records. Add the rocket_name from the matched rocket record to the launch record. (This is conceptually similar to a left join, keeping all launch records).
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with the corresponding rocket_name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable: 
	LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
	ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval (for launches, for rockets), number of launches retrieved, number of rockets retrieved, starting join operation, join operation complete, number of records after join, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + join operation). Print this time after the join operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting joined list of dictionaries from the join operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting joined JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
# Databricks notebook source
import requests
import json
import logging
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define API endpoint URLs
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
HTTPBIN_UPLOAD_URL = "https://httpbin.org/post"

def fetch_data_from_api(url):
    """
    Fetches data from a given API endpoint and handles potential errors.

    Args:
        url (str): The URL of the API endpoint.

    Returns:
        list or None: A list of dictionaries containing the API response data,
                       or None if an error occurred.
    """
    try:
        logging.info(f"Starting data retrieval from {url}")
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        data = response.json()
        logging.info(f"Successfully retrieved {len(data)} records from {url.split('/')[-1]}")
        return data
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during API call to {url}: {e}")
        return None

def join_launches_and_rockets(launches_data, rockets_data):
    """
    Performs a JOIN operation to combine launch and rocket data.

    Args:
        launches_data (list): A list of dictionaries representing launch data.
        rockets_data (list): A list of dictionaries representing rocket data.

    Returns:
        list: A list of dictionaries, where each dictionary represents a launch
              enriched with the corresponding rocket_name.
    """
    if not launches_data or not rockets_data:
        logging.warning("One or both of the input data lists are empty. Skipping join operation.")
        return launches_data if launches_data else []

    logging.info("Starting join operation between launches and rockets data.")
    rocket_map = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets_data}
    joined_data = []
    for launch in launches_data:
        rocket_id = launch.get('rocket', {}).get('rocket_id')
        rocket_name = rocket_map.get(rocket_id)
        launch['rocket_name'] = rocket_name
        joined_data.append(launch)

    logging.info("Join operation complete.")
    logging.info(f"Number of records after join: {len(joined_data)}")
    return joined_data

def upload_data(data):
    """
    Uploads the given data to the httpbin.org/post endpoint.

    Args:
        data (list): The data to be uploaded (list of dictionaries).
    """
    if not data:
        logging.warning("No data to upload.")
        return

    logging.info(f"Starting upload of {len(data)} records to {HTTPBIN_UPLOAD_URL}")
    try:
        headers = {'Content-Type': 'application/json'}
        json_data = json.dumps(data)
        response = requests.post(HTTPBIN_UPLOAD_URL, headers=headers, data=json_data, timeout=10)
        response.raise_for_status()
        logging.info(f"Upload to {HTTPBIN_UPLOAD_URL} successful. Status code: {response.status_code}")
        logging.debug(f"Response from httpbin: {response.json()}")
        logging.info("Upload outcome: Success")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during upload to {HTTPBIN_UPLOAD_URL}: {e}")
        logging.info("Upload outcome: Failure")

if __name__ == "__main__":
    total_start_time = time.time()

    # Retrieve data from SpaceX API
    launches_data = fetch_data_from_api(LAUNCHES_API_URL)
    rockets_data = fetch_data_from_api(ROCKETS_API_URL)

    # Perform join operation
    join_start_time = time.time()
    joined_data = join_launches_and_rockets(launches_data, rockets_data)
    join_end_time = time.time()
    join_execution_time = join_end_time - join_start_time
    logging.info(f"Time taken for data retrieval and join operation: {join_execution_time:.2f} seconds")

    # Upload the result
    upload_data(joined_data)

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    print(f"Total script execution time: {total_execution_time:.2f} seconds")